In [1]:
# first set global root data directory
import pylabs
pylabs.datadir.target = 'jaba'
from pathlib import *
import pandas as pd
import numpy as np
from pylabs.io.mixed import df2h5, h52df, get_h5_keys
from pylabs.utils import *
from pylabs.projects.genz.file_names import project, Optsd
import qgrid
qgrid.enable
import ipywidgets as widgets
fs = Path(getnetworkdataroot())
opts = Optsd()
style = {'description_width': 'initial'}
input_selkey = widgets.Dropdown(options=sorted(get_h5_keys(opts.info_fname, key='auto_qc')), disabled=False,
    description='Subject DWI to qc:', style=style)
input_selkey.layout.width = '350px'
input_selkey

/home/toddr/Software/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
setting root data directory to jaba.


Dropdown(description=u'Subject DWI to qc:', layout=Layout(width=u'350px'), options=('/sub-genz103/ses-1/dwi/auto_qc', '/sub-genz104/ses-1/dwi/auto_qc', '/sub-genz201/ses-1/dwi/auto_qc', '/sub-genz203/ses-1/dwi/auto_qc', '/sub-genz205/ses-1/dwi/auto_qc', '/sub-genz301/ses-1/dwi/auto_qc', '/sub-genz302/ses-1/dwi/auto_qc', '/sub-genz303/ses-1/dwi/auto_qc', '/sub-genz305/ses-1/dwi/auto_qc', '/sub-genz308/ses-1/dwi/auto_qc', '/sub-genz501/ses-1/dwi/auto_qc', '/sub-genz502/ses-1/dwi/auto_qc', '/sub-genz503/ses-1/dwi/auto_qc', '/sub-genz506/ses-1/dwi/auto_qc', '/sub-genz508/ses-1/dwi/auto_qc', '/sub-genz510/ses-1/dwi/auto_qc', '/sub-genz906/ses-1/dwi/auto_qc'), style=DescriptionStyle(description_width=u'initial'), value='/sub-genz103/ses-1/dwi/auto_qc')

In [2]:
pick = {'subj': input_selkey.value.split('/')[1], 'session': input_selkey.value.split('/')[2], }
# new h52df function makes all cols numeric. yes!
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
pick['dwi_fname'] = all_dwi_auto_qc.loc[0, 'dwi_fname']
# todo: make python scriptlet to load files into FSLeyes
# fslv_names = ' '.join([x + ' -b 1,200' for x in all_dwi_auto_qc.loc[0, ['dwi_fname', 'topup_fname', 'topdn_fname']].values])
# with WorkingContext(str(Path(all_dwi_auto_qc.loc[0, 'dwi_fname']).parent)):
#     with open('fslview_cmd.sh', 'w') as f:
#         f.write('fslview ' + fslv_names + ' &\n')

# limit to only active rows with mask
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] != -9999, 'itopup_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] != -9999, 'itopdn_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] == -9999, 'itopup_visqc'] = False
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] == -9999, 'itopdn_visqc'] = False
all_dwi_auto_qc.loc[:, 'dwi_visqc'] = True
with WorkingContext(str(Path(pick['dwi_fname']).parent/'qc')):  
    topup_gvolspng = open('{subj}_{session}_topup8b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topup_bvolspng = open('{subj}_{session}_topup8b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    topdn_gvolspng = open('{subj}_{session}_topdn7b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topdn_bvolspng = open('{subj}_{session}_topdn7b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi800_gvolspng = open('{subj}_{session}_b800-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi800_bvolspng= open('{subj}_{session}_b800-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi2000_gvolspng = open('{subj}_{session}_b2000-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi2000_bvolspng= open('{subj}_{session}_b2000-qc_bad_plot.png'.format(**pick), 'rb').read()

dwicols = [u'bvals', u'auto_dwi_vol_idx', u'auto_dwi_qc', 'dwi_visqc', u'x_bvec', u'y_bvec', u'z_bvec']
topudcols = [u'itopup', u'alltopup_idx', u'topup_qc', 'itopup_visqc', u'itopdn', u'topdn_qc', 'itopdn_visqc',]
dwi_auto_qc = all_dwi_auto_qc[dwicols]
topud_auto_qc = all_dwi_auto_qc[topudcols]
dwi_w = qgrid.show_grid(dwi_auto_qc, show_toolbar=False)
topupdn_w = qgrid.show_grid(topud_auto_qc, show_toolbar=False)

In [3]:
items = [widgets.Image(description='dwi b2000 bad vols', value=dwi2000_bvolspng,    format='png',    width=500,    height=600),
        widgets.Image(description='dwi b2000 good vols', value=dwi2000_gvolspng,    format='png',    width=500,    height=600),]
#items_layout = [widgets.Layout(flex-basis='50%', ), widgets.Layout(flex-basis='50%', widgets.Layout(flex-basis='100%',)]
box_layout = widgets.Layout(display='flex', align_items='stretch', border='solid', width='100%')
top_box = widgets.HBox(children=items)
bottom_box = widgets.HBox(children=[dwi_w])
dwi_box = widgets.Box([top_box, bottom_box], layout=box_layout)
dwi_box

Box(children=(HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xbf\xbf\xbf\x1f\x1f\x1f___???\xdf\xdf\xdf\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xf5Uk\xc9\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\x9d\x8br\x9b<\xb7@\x0f7\xbd\x1c\xd3d\xd2\xa6I\xff\\\x9a\xf6\xeb\xc0\xfb?\xc2A\xe8\x82$\x04\x86\x04\xb0e\xaf5\xd3\xc6\xc6\xc0\x16\x18\x96\xb7\x84\x10\xff\xd7\x02\x00$\xc2\xff\x9d\xbb\x00\x00\x00KAX\x00\x90\x0c\x08\x0b\x00\x92\x01a\x01@2 ,\x00H\x06\x84\x05\x00\xc9\x80\xb0\x00 \x19\x10\x16\x00$\xc3\xed\x08\xabi\x8as\x17\xe1x\x1aCQ\x94\xb9\x9aT7!\xa5\x9c\\4Md\xf9\xac,\n\xb9l\x16L\xcf\xeb\xaa\x9b^\x8d\xa6G\x88\xaf\xf8@\x8aa\'\x84\x05\xce\x1a\xff\xa8\x88\xec\xae\xb5\xa1N\xcc!\xcab\xf1qx\x93G\xec)\x10\xd6\xb61\x86C\xdeL\x13\xeeA\x9c\xcb\xe3\xb5Zp\x9a\xafFD\xcf\x15_LBNZ!\xac\xda9\xd7kgz^\r\xd3On\xcbN\xc2\xca\x941\xc50\xa1\xec\xa6\xc4\x14Zx\x1b[\t\xe7\xa3~;\x1c3Ev\xd7*Nn\xab\xe8\xd7\xbcpm\xd1#v\xb4\xdd\x91cj\xc9\x94TAX\x1b"b\xc2\xca\x9c\x03\xd4\x9c\xe8\x95\xd8<v6%\xac\xb2\xa7\x8f\\\xc8\xb0\x99\x9aP\xda\x8f\xd49>>\xd7D\xa5\xb3\x92\xaa\xf0\xcb\x9cy\xd3\xebv\x9e]\x84\x95[\x0b\xd5na\xa3\xfb\xb6\xb0\x1b\xda\x17\xd89\n\xc4\xe0kEdw\xad\xe2\xe4\xb6v{\xbd\xaa\x97z#r\xc4\x8e\xb6;vL-\x99\x92,\x08kCry\x1ek\xcci \x9c\x83X\x1e7e]6;\x14E\xc4\xcf\x15W\x9c\xdd\x1c\x95\xf3Q\xb0\xc0hy\xb9\xc6\xa6\x16\xeau\xed\x9c\xbf\xb9<\xf2UZ\xd2\xa7Z\'\xeaN{\x08+o\xd4\x9e,\xacodA\xaaZ\xed[1]\x00\xe9\xda\xcc}Wz\xdf\xc5\xcc\xeeZ\xc4\xc9m\xedf\x10\xf3s8\x8c\x0f\x93\xf1vG\x8e\xa9%S\xd2\x05amH6>{\xe5q\xdf\x0c\x1f\xf7\'\x93\xcc\x06N\xe5%k#;a\\\xdc\x8d\x96\x06r\xc2\x9e\x12V\xe5\x9d\xfab8\x7f\xbd\xd5\x94\'O\xeb=\x84U\x99d!\xd3\xc2\x94\xbb\xb6\xdf\xf5\x91}\xeb\x15 s3\xaan^\xa9\xdc\xe1;\x9b\xd9]\x8bX"\xac\xe5k\x1b\x1f\xb1\xa3\xed\x8e\x1cSK\xa6$\x0c\xc2\xda\x90:\xf8\xfd,u\x02\xaf\xdf\x16\xe6\xdc\x10q\xbb|\x96 \x8c\x8b\xb7\xd1\xb9\xf7\xee\x84\xb0\xb2 U\x91\xe7o\x9f\x9b\xd4^%J\xd6\xa2\xdc\xd9\xc6\xec ,gCtiJ\xab\x1d1\xfa\x9e\xbd\x02\x08G\xb0\xfd\xac\x9e\xc1fv\xd7"v\x16\xd6x\xbb#\xc7\xd4\x92)\tsE\xc2\xd2\xcd\x91\xfa\xc0u\xbem\xf5R\xfe/d3r\xa5\xeb9\x01\xb5\xfc\xf9\xaa\xddcJ\xad\xd0\xd6 \xfa\x0fT\xeb\xa5\x8d\xe1\xd0\xf6\xa7\x8d\xb7\xca\xa2qM"\xbc\x1c=l\xc8\xf0K?\x8e\x1e\x99#\x1e\xc6\xc3?\xe6\xbdl\xe2\x84\xb0\x8a0[4gKP\xab)OmJ\xbfb5)\xdc\xb8\xda\xaeB}\x92Y\xfb\xd5:\x17\x88os=d\nz\xa7VC\xe9GN\x98\x14\x96Z\x8d\xbb\xe3fv\x97\x17\xd5n\xb5\xb7\x1dN\xa8x\xb9\x1b\xf7\x8b\x8a-\xda%\xca\x99;{p\xc4\x8e\xb7{|L-\x99\xe2\xa3C\xcc\x1c\xf9\x97\xc4\xd5\x08kh\x8eT?\x981a\x99y\x8a\xf1Io?\xb2\x8dA\xc2\xac\xd0$\x1a\xf2\x83:\x88\xe1\x0b\xab\n~\x12\xf3\xacc8\x88\x87\xc3-\x0bs\xf3\xb0\xf4\xe3\xe8\xa39\xa6\xc2x\xf8g\xa0\x17v^X\xf9\xb8\xaa\xa7\xce\xdf<(\x80\xc8s1\xbb)r\xc5\xd5\xd4\xc6\x99j\x8d*\x96le\xc9\xdcX\x13\xdb\\:&Q\xe5\x1e\xda\x0c\xdbQ\xb7\x01o\xcbj\xe7\x94U\xeau\x8d;\xb3\xbb\xfc\xdcM}\xdf\xc1v\xd8PS\xe5v\x0e\x95\xe8\xa2u\xd3\x04\xc2\xf2\x8f\xd8\xf1v\x8f\x8f\xa9%S<\xe6\x8e\xfc

In [4]:
items = [widgets.Image(description='topup bad vols', value=topup_bvolspng,    format='png',    width=400,    height=600),
        #widgets.Image(description='topup good vols', value=topup_gvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn bad vols', value=topdn_bvolspng,    format='png',    width=400,    height=600),
        #widgets.Image(description='topdn good vols', value=topdn_gvolspng,    format='png',    width=400,    height=600),
]
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
topupdn_box = widgets.Box(children=items+[topupdn_w], layout=box_layout)
topupdn_box

Box(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xdf\xdf\xdf___???\xbf\xbf\xbf\x1f\x1f\x1f\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xa8*\xe1\x90\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\xdd\t\x96\xa2J\xb7\x06\xd0G\x17\x93c\x18\xff\xc2\xf9\x0f\xe1I\x13H\xa7)U&Y\'\xef\xdek\xdd[\xa9"\x07\x10>!\x08\xf0\xffn\x00A\xfc\xdfOO\x00\xc0\xbb\x04\x16\x10\x86\xc0\x02\xc2\x10X@\x18\x02\x0b\x08C`\x01a\x08, \x0c\x81\x05\x84\xf1\x9b\x03\xab\xeb\xd2OO\xc2\xf5\xba,\xa5\xaa\x19\x9fj\xbb\xad\xaa\x7f:u\xdd\xc1\xfb\x8b*\xa5\xfe\xbd\xc5\xe6\xf9\xa6\xad\xef\xcf\xd7\xbb\xe7\x0f\x1c\x8f\xf8\xdb\xa4\xc7,o\'\xaf\xe8\xd6\xeb\xc0\xc1\xc29[\xea\x8b!\xca*\xad*\xfe\xb0\xdf\xb7\t\x08\xac\xbf\xad\xf1\xd8\x08\xf2s\xe5r\xb5n\xfa5\xb8~c3?\xad<\xdcz\xd6\xc1T\xf6O\x9d\x08\xacv\xb1\xf5\xb7\x8b\xe7\x9b\xfa\xf1\xfc\x97\xf3\xf2M\x81\xf5lI\xa6\xd5\xac\xd5\xe5\xe2\xa5a\xaa\x17\xc9t\xb0pN\xf9r\xce\xcaa\xcco\x8e\xad\xad\xaa\xd3S\xf0\x95\xa6\xaa\xfb\xdc\xce\xf3|\xb8\t|\xdf*y\x01\x81\xf5W\xca\xa3\xc0*\x16\xabl}\xb4\x1d}F\xf1,\xb0\xaa\xc1P9\xf5e\x8b\xf1\x89j~i\xdc\x0f\xd9o}e=\xed\xa7\xd4i=\xcd\xc5\xea\xf9\xf6\xf6\xda\xf7\x04\xd6\xd3%\x99\xe6\xd9\x1a&/=^)\x1f\xe9<:X8\xa7|9g\xf7e\\\xb7\xefF\xc1\xe7\x97S9\x7f\xafLK\xe9h\x13\xf8\xc6U\xf2\x02\x02\xeb\xaf4\xfdv<\xc9\x1bF\xb9X\x11\xfb\x95\xa3j\xab\xee\x1b&\xa5<^\xdf\x97\xc1y\x1f\xa2^\xbc\xb4y\xc3\xee\xfd\xfd\x18\xbbv\\\x8d\xcbv\xb1E7\xfd\xea=~i\x0f\xbbZ_\x1cM}K`=_\x92\xcbr}\xb2\x16\xcbG\xd5j\xf8\x17\x0b\xe7-_\xce\xd9}\x80\xf7c\xe0\xe3\xcbi\xf8\x00\xef\x0b\xa9_V\xe3\xbc\x1d\xacw\xdf\xb8J^A`\xfd\x95b\xbf\xf5\xf6[B\xf7xy\xd8\xe8\xfbo\xbe\xaf\xf6K\xceV^\x94YZ\xcet\xbf\x02/\xca~\x15X\xf5j\xa7\xa3|l\xd1\xab\xd1T_n\xe8\xdf\x11X/\x96\xe4\xaa\\\xb1\xdc\xa3\xba\x0f\xdb\x07\xec\xe3\x13z\xb1p\xde\xf2N`}plgUy\xb7i\x9e\xb7\xfd&\xf0\x8d\xab\xe4%\x04\xd6_i7\xdf\xa8\xd5\xd4\xa02=Lyk)\xdfo\xd9x\xc7\xa6\xcc\xd2j\xa6\x9b\xd5\xa3/\x02\xab\xe8\xd6\x07I\xfdZ?\xec\xad\xb4\xab\xc3\xaa\xfe\xb8\xea\xf5^\xc4w\x04\xd6\x8b%\xb9*W.\xe2\xb4\xec\xe7~\x95`/\x16\xce\xbbS\xf1\x97\x03\xfc\xf1\xc0o\x8e\xf0\xb1O<,\x86\xc3\xfd\xd1oY%\xaf\x12:\xb0\x8a\xf1\xc4\xd5\xf4\r\xba\xf8p\xc6?\xfb\xff\x97}3r\xdd\x1enam\xdd\xbf\xb4\\q\xc6\x11\xce\xc7\x14\xc3\x0bc\x13\xe5\\c\xe16|\xa7\xadF\x99\xbae\x92\x94\x8fI\xaa\x97\x87*GS\xbf\xaf~0\xc4q\x99\x95\xf5*\xba\xda\xbf\xf8"\xb0R\xb7\xd9[\xcc[\xf4\xe68\xa7\xfajV\x86\x11\x8fOmg\xae\x9dG1\xbeR\xcc\xe9\xd7N_\xf8\xc7\xf3\xfcjI>\r\xacq\x94\xcb\xc5\xf4b\xe1<\xa6`9\x8f\xab\xa9^\x94:\x9e\xcan\xf9\xb1\x1c\xbd\xf5\xbe[\\\x1c\x0e\xbc\x1dz\x98\xd0q\xfd,w\x93>\xfd\xf9j\x98r\x11X\xabM\xe0\xf5*\x99\x87}\xb1M\xfc\xb4\xc0\x81\xd5\xcc\xa7\x87\xc6\xaf\xd0\xa3\xc0\xca\xc3\xa4\xfdF?\xbf47\x06\x95y\x84yG\xa3\x7f\xa1\xdd\xd4X\x07V\xfd\xa89\x8e\xb4\xb8{\xac\xd6\x8fM\xa0\xd8\xee\x80o\xa7~_}7\xc4\xb32+\xebmrU\xf6u`5\xfbC\xbdq\x8bn6\x13P6\xcd:\xff\xb7\x13\xda\x8f\xb8~6s\xcdj\xb2\xfa\xa6\x94bY\xeb\xc9<\xbfZ\x92\xab\xf9h\x17[\xe1\x18\xb4\x

In [5]:
dwi_w.get_changed_df()
topupdn_w.get_changed_df()
all_dwi_visqc = pd.concat([dwi_w.get_changed_df(), topupdn_w.get_changed_df()], axis=1)
df2h5(all_dwi_visqc, opts.info_fname, input_selkey.value.replace('auto_qc', 'vis_qc'))

In [5]:
topupdn_w.get_changed_df()

,itopup,alltopup_idx,topup_qc,itopup_visqc,itopdn,topdn_qc,itopdn_visqc
0,0,0,0,True,0,0,True
1,1,1,0,True,1,0,True
2,2,2,0,True,2,0,True
3,3,3,0,True,3,0,True
4,4,4,0,True,4,0,True
5,5,5,0,True,5,0,True
6,6,6,0,True,6,0,True
7,-9999,7,0,False,-9999,-9999,False
8,-9999,-9999,-9999,False,-9999,-9999,False
9,-9999,-9999,-9999,False,-9999,-9999,False


In [7]:
dwi_w.get_changed_df()

,bvals,auto_dwi_vol_idx,auto_dwi_qc,dwi_visqc,x_bvec,y_bvec,z_bvec
0,0,0,0,True,0.000,0.000,0.000
1,2000,0,1,True,1.000,0.000,0.000
2,2000,1,0,True,0.000,1.000,0.000
3,2000,2,0,True,0.000,0.000,1.000
4,2000,3,0,True,-0.625,-0.777,0.076
5,2000,4,0,True,-0.745,0.623,0.237
6,2000,5,1,True,0.025,-0.095,0.995
7,2000,6,0,True,-0.853,-0.004,-0.522
8,2000,7,1,True,-0.155,0.904,-0.399
9,2000,8,1,True,-0.025,-0.716,-0.698
